# LATAM Airlines

Scrapear el sitio de Despegar para averiguar datos de vuelos en función del **origen** y **destino**, **fecha** y **cabina**. La información que esperamos obtener de cada vuelo es:

- Precios(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**NOTA**: en caso de no poder acceder (status_code = 403), buscar un sitio similar.

www.edestinos.com

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#url = 'https://www.edestinos.com/flights/select/oneway/ap/mvd/ap/for?departureDate=2020-12-10&pa=1&py=0&pc=0&pi=0&sc=economy'

In [2]:
url = 'https://www.latam.com/es_uy/apps/personas/booking?fecha1_dia=04&fecha1_anomes=2020-11&from_city1=MVD&to_city1=MAD&ida_vuelta=ida_vuelta&fecha2_dia=21&fecha2_anomes=2020-11&from_city2=MAD&to_city2=MVD&cabina=Y&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'

In [3]:
r = requests.get(url)

In [4]:
r.status_code

403

In [5]:
soup = BeautifulSoup(r.text, 'lxml')

In [6]:
# Importar Selenium (webdriver)

from selenium import webdriver

In [7]:
# Iniciar navegador automatico
options = webdriver.ChromeOptions()
options.add_argument('--incognito')

driver = webdriver.Chrome(executable_path='/Users/Martin/Documents/Proyectos/Airlines_scrapper/chromedriver', options=options)

In [8]:
driver.get(url)

In [9]:
# Extraer informacion de la pagina

vuelos = driver.find_elements_by_xpath('//li[@class="flight"]') # // buscar en todo el arbol
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="289a0012-d65a-4765-aa1a-29cf69533b49")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="8344d7e2-3b00-4ce6-a1d7-b72489b457fe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="c703a831-e582-4ee3-be3e-7ae5ae6933de")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="3044630d-61ae-440f-8b49-480c0bd42e10")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="0113042b-0032-4b29-b558-e8f76f926443")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="61f153e3-e019-48f2-afe6-fae3ed74fa88")>]

In [10]:
vuelo = vuelos[0]

In [11]:
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="289a0012-d65a-4765-aa1a-29cf69533b49")>

**Extrayendo horarios y duración**

In [12]:
#Hora de partida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime') # . buscar desde allí hacia abajo

'11:18'

In [13]:
#Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')


'12:50'

In [14]:
# Duración de vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT21H32M'

**Contando escalas e interactuando con elementos del sitio**

In [15]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [16]:
boton_escalas.click() #click en boton

In [17]:
# Segmentos del contenido en donde se ven las escalas
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
# Sin '.' porque busco todo el album.
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="9954ced3-cb9c-43b9-b32f-e3d3d2571754")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="6a83d98c-81de-4fb6-b07b-da3eae7ba9b8")>]

In [18]:
escalas = len(segmentos) - 1 # Porque cuenta desde salida/escala y escala/salida
print(f'El vuelo de MVD a MAD tiene {escalas} escala(s).')

El vuelo de MVD a MAD tiene 1 escala(s).


**Desafío: Escalas**

- **Escala #1**

In [19]:
segmento = segmentos[0]

In [20]:
# Aeropuerto de salida
segmento.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 


'Carrasco Intl.'

In [21]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'11:18'

In [22]:
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/span[@class="sr-only"]').text

'Salida a las 11 Horas 18 Minutos, de Carrasco Intl.'

In [23]:
# Duración de vuelo a próxima escala
segmento.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')

'2:32'

In [61]:
try:    
    combinacion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-label sc-hORach NXcGo"]').text
    cambio_avion = segmento.find_element_by_xpath('//div[@class="sc-hMFtBS cfwWiO"]/span[@class="connection-changes sc-hORach NXcGo"]').text
    print(f'{combinacion} -- {cambio_avion}')
except:
    print('Revise XPath.')
    pass



Conexión en Santiago de Chile -- (cambio de avión)


In [60]:
modelo_avion = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
flota = segmento.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text

print(f'Flota {flota} - {modelo_avion}')

Flota LA405 - Airbus 320-200


- **Escala #2**

In [48]:
segmento_2 = segmentos[1]

In [49]:
# Aeropuerto de salida
segmento_2.find_element_by_xpath('.//div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text 

'A. Merino Benítez Intl.'

In [50]:
# Hora de salida
segmento_2.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'19:55'

In [51]:
segmento_2.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/span[@class="sr-only"]').text


'Salida a las 19 Horas 55 Minutos, de A. Merino Benítez Intl.'

In [52]:
# Duración de vuelo a próxima escala
segmento_2.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')

'12:55'

In [59]:
modelo_avion = segmento_2.find_element_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
flota = segmento_2.find_element_by_xpath('//div[@class="airline-flight-details"]/b').text

print(f'Flota {flota} - {modelo_avion}')

Flota LA405 - Airbus 320-200


- **Tarifas**

In [62]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

In [63]:
vuelo.click()

In [70]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]//span[@class="price"]')
tarifas

[<selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="3b9cee46-2f15-4e7e-a003-936fc29cf4dc")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="21fc13a0-f116-4fd9-8f6e-1bf58d3e7e64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="74703bce7a5a8fa0b7129a167262ed99", element="7a88cfff-4691-441c-87a4-ecaaf80b37ad")>]

In [73]:
for tarifa in tarifas:
    print(tarifa.text)

US$332
US$385
US$445
